<a href="https://colab.research.google.com/github/qwasd34/DEV_DATA/blob/main/%5B%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C%5D_%E1%84%92%E1%85%B2%E1%84%83%E1%85%A2%E1%84%91%E1%85%A9%E1%86%AB_%E1%84%8E%E1%85%AE%E1%84%8E%E1%85%A5%E1%86%AB_%E1%84%89%E1%85%B5%E1%84%89%E1%85%B3%E1%84%90%E1%85%A6%E1%86%B7_%E1%84%89%E1%85%B5%E1%86%AF%E1%84%89%E1%85%B3%E1%86%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 콘텐츠 기반 필터링

- ratingd 평가
- users 평가를 내린 사용자들의 인구통계
- data 핸드폰 종류에 따른 특성들


프로파일 구성
- 브랜드, 모델, os , 메모리, ram , 가격
- Embedding 계획
  - 브랜드 , 모델, os : 텍스트 -> TF-IDF점수 적용
    - 각각각들의 상대적인 중요도 ..
  - 메모리 ram 가격 : 숫자 -> Minmax scaling
    - 숫자 분포를 보면 들쑥날쑥함

## 데이터 로드 및 프로파일 구성

In [1]:
import pandas as pd

# 데이터 로드
data_path = 'cellphones data.csv'
data_df = pd.read_csv(data_path)

In [2]:
# 총 33개의 핸드폰 종류가 존재
data_df

,cellphone_id,brand,model,operating system,internal memory,RAM,performance,main camera,selfie camera,battery size,screen size,weight,price,release date
0,0,Apple,iPhone SE (2022),iOS,128,4,7.23,12,7,2018,4.7,144,429,18/03/2022
1,1,Apple,iPhone 13 Mini,iOS,128,4,7.72,12,12,2438,5.4,141,699,24/09/2021
2,2,Apple,iPhone 13,iOS,128,4,7.75,12,12,3240,6.1,174,699,24/09/2021
3,3,Apple,iPhone 13 Pro,iOS,256,6,7.94,12,12,3065,6.1,204,999,24/09/2021
4,4,Apple,iPhone 13 Pro Max,iOS,256,6,8.01,12,12,4352,6.7,240,1199,24/09/2021
5,5,Apple,iPhone XR,iOS,64,3,4.22,12,7,2942,6.1,194,236,26/10/2018
6,6,Asus,Zenfone 8,Android,128,8,6.76,64,12,4000,5.9,169,599,12/05/2021
7,7,Samsung,Galaxy A13,Android,32,3,1.36,50,8,5000,6.6,196,154,23/03/2022
8,8,Samsung,Galaxy A32,Android,64,4,2.20,48,13,5000,6.5,205,199,22/01/2021
9,9,Samsung,Galaxy A53,Android,128,6,3.79,64,32,5000,6.5,189,312,24/03/2022


TF-IDF 하려면 한 뭉치로 묶어야함 ... 띄어쓰기로 묶어서 feat text ! 컬럼으로 만들어

In [3]:
# 프로파일 구성을 위한 특성 선택
# 선택한 특성: 'brand', 'model', 'operating system', 'internal memory', 'RAM', 'price'

# 'brand', 'model', 'operating system'을 기반으로 문자열 생성 : TF-IDF에 입력으로 들어갈 예정
data_df['feat_text'] = data_df[['brand', 'model', 'operating system']].astype(str).agg(' '.join, axis=1)
# 'internal memory', 'RAM', 'price'은 숫자형 특성 : MinMax Scailing에 활용 예정
data_df['feat_num_IM'] = data_df['internal memory']
data_df['feat_num_RAM'] = data_df['RAM']
data_df['feat_num_PRC'] = data_df['price']

data_df.head()

,cellphone_id,brand,model,operating system,internal memory,RAM,performance,main camera,selfie camera,battery size,screen size,weight,price,release date,feat_text,feat_num_IM,feat_num_RAM,feat_num_PRC
0,0,Apple,iPhone SE (2022),iOS,128,4,7.23,12,7,2018,4.7,144,429,18/03/2022,Apple iPhone SE (2022) iOS,128,4,429
1,1,Apple,iPhone 13 Mini,iOS,128,4,7.72,12,12,2438,5.4,141,699,24/09/2021,Apple iPhone 13 Mini iOS,128,4,699
2,2,Apple,iPhone 13,iOS,128,4,7.75,12,12,3240,6.1,174,699,24/09/2021,Apple iPhone 13 iOS,128,4,699
3,3,Apple,iPhone 13 Pro,iOS,256,6,7.94,12,12,3065,6.1,204,999,24/09/2021,Apple iPhone 13 Pro iOS,256,6,999
4,4,Apple,iPhone 13 Pro Max,iOS,256,6,8.01,12,12,4352,6.7,240,1199,24/09/2021,Apple iPhone 13 Pro Max iOS,256,6,1199


## 각 특성 당 정보 추출

In [4]:
# Embedding으로 특성 표현
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

# TF-IDF Vectorizer를 사용하여 특성 문자열을 벡터로 변환
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data_df['feat_text'])

# MinMaxScaler를 활용해 수치형 데이터를 정규화
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(data_df[['feat_num_IM', 'feat_num_RAM', 'feat_num_PRC']])

In [5]:
# tfidf_matrix를 dense 형태로 변환하고, 처음 5개 행을 확인
tfidf_matrix_dense = tfidf_matrix.toarray()
print(tfidf_matrix_dense[:5])  # 처음 5개 행만 출력

feature_names = tfidf_vectorizer.get_feature_names_out()
# 특성(단어) 이름을 출력
print(feature_names)

[[0.         0.         0.         0.         0.         0.
  0.         0.47055875 0.         0.         0.         0.
  0.         0.         0.38669679 0.         0.         0.
  0.         0.         0.         0.         0.38669679 0.38669679
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.57443123 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.443908
  0.         0.         0.         0.         0.         0.
  0.         0.         0.39270242 0.         0.         0.
  0.         0.         0.         0.         0.39270242 0.39270242
  0.         0.58335249 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.     

첫번째 핸드폰에 8번째가 값이있어 ! -> 2022 라는 값이 있는것


메모리 ,램 , 가격 -> 범위적인 이슈는 없어짐

In [6]:
print(scaled_features)
print(scaled_features.shape)

[[0.2        0.11111111 0.16051364]
 [0.2        0.11111111 0.30497592]
 [0.2        0.11111111 0.30497592]
 [0.46666667 0.33333333 0.46548957]
 [0.46666667 0.33333333 0.57249866]
 [0.06666667 0.         0.05724987]
 [0.2        0.55555556 0.25147138]
 [0.         0.         0.01337614]
 [0.06666667 0.11111111 0.03745318]
 [0.2        0.33333333 0.09791332]
 [0.2        0.55555556 0.21348315]
 [0.2        0.55555556 0.41198502]
 [0.2        0.55555556 0.38041734]
 [0.2        0.55555556 0.41198502]
 [0.46666667 1.         0.89352595]
 [0.2        0.55555556 0.19796683]
 [0.2        0.33333333 0.17121455]
 [0.2        1.         0.30497592]
 [0.2        0.33333333 0.09095773]
 [0.2        0.55555556 0.13376137]
 [0.2        0.55555556 0.34831461]
 [0.2        0.55555556 0.27822365]
 [0.46666667 1.         0.45906902]
 [0.46666667 0.55555556 0.39753879]
 [0.2        0.11111111 0.02407705]
 [0.46666667 0.55555556 0.16372392]
 [0.2        0.55555556 0.26163724]
 [0.2        0.55555556 0.15

## 아이템의 프로파일을 생성

In [7]:
# TF-IDF 벡터와 스케일링된 수치형 특성을 결합
import numpy as np
combined_features = np.hstack((tfidf_matrix.toarray(), scaled_features))

In [8]:
combined_features

array([[0.        , 0.        , 0.        , ..., 0.2       , 0.11111111,
        0.16051364],
       [0.        , 0.        , 0.        , ..., 0.2       , 0.11111111,
        0.30497592],
       [0.        , 0.        , 0.        , ..., 0.2       , 0.11111111,
        0.30497592],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.01605136],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.06666667, 0.11111111,
        0.03210273]])

## 유사도 계산 및 추천

In [9]:
# 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

# 코사인 유사도를 사용하여 항목 간 유사도 계산
cosine_sim = cosine_similarity(combined_features, combined_features)

In [10]:
cosine_sim

array([[1.        , 0.50113071, 0.59591399, ..., 0.00248105, 0.        ,
        0.24454722],
       [0.50113071, 1.        , 0.83605019, ..., 0.00457353, 0.        ,
        0.03285111],
       [0.59591399, 0.83605019, 1.        , ..., 0.00457353, 0.        ,
        0.03285111],
       ...,
       [0.00248105, 0.00457353, 0.00457353, ..., 1.        , 0.46320185,
        0.40357208],
       [0.        , 0.        , 0.        , ..., 0.46320185, 1.        ,
        0.48431097],
       [0.24454722, 0.03285111, 0.03285111, ..., 0.40357208, 0.48431097,
        1.        ]])

In [12]:
cosine_sim.shape

(33, 33)

핸드폰 - 핸드폰 사이의 유사도 계산

In [13]:
# 추천 생성 예시
# 특정 휴대폰과 비슷한 휴대폰을 번환하기 위한 함수 구현
def get_content_based_recommendations(cellphone_index, cosine_sim=cosine_sim, data_df=data_df):
    sim_scores = list(enumerate(cosine_sim[cellphone_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) #유사도가큰것 순서
    sim_scores = sim_scores[1:6]  # 가장 유사한 상위 5개 항목 선택, 0: 자기 자신 (가장큰것)
    cellphone_indices = [i[0] for i in sim_scores]
    return data_df.iloc[cellphone_indices][['brand', 'model', 'operating system', 'internal memory', 'RAM', 'price']]

In [14]:
# 원하는 N 번째의 휴대폰에 대한 추천 생성 예시 실행
from IPython.display import display

N = 4
print('선택한 휴대폰 : ')
display(data_df.iloc[N:N+1][['brand', 'model', 'operating system', 'internal memory', 'RAM', 'price']])

example_recommendations = get_content_based_recommendations(N)
print('\n추천된 비슷한 휴대폰 : ')
display(example_recommendations)

선택한 휴대폰 : 


,brand,model,operating system,internal memory,RAM,price
4,Apple,iPhone 13 Pro Max,iOS,256,6,1199



추천된 비슷한 휴대폰 : 


,brand,model,operating system,internal memory,RAM,price
3,Apple,iPhone 13 Pro,iOS,256,6,999
2,Apple,iPhone 13,iOS,128,4,699
1,Apple,iPhone 13 Mini,iOS,128,4,699
28,Sony,Xperia Pro,Android,512,12,1998
0,Apple,iPhone SE (2022),iOS,128,4,429


# 사용자 기반 협업 필터링

## 데이터 준비
- 사용자들 rating  파일사용 (상호작용 데이터)
- 전제 33개중 10개를 제시 -> 23개는 정보없음
  - 0으로 채워넣기
- 행에는 사용자 정보, 열에는 핸드폰 정보로하는 matrix 생성
  - DF.pivot_table: 기존 DF 에서 필요한 정ㅇ보를 요약, 추출해 새로운 DF 생성
  - 같은 사용자 ID를 기준으로 rating column값을 집계

In [15]:
import pandas as pd

user_item_df = pd.read_csv('cellphones ratings.csv')
user_item_df

,user_id,cellphone_id,rating
0,0,30,1
1,0,5,3
2,0,10,9
3,0,9,3
4,0,23,2
...,...,...,...
985,258,31,5
986,258,17,8
987,258,23,9
988,258,27,8


## 사용자(행) - 아이템(열) 상호 작용 테이블 생성

In [16]:
# 사용자 기반 협업 필터링을 위한 사용자-항목 평가 행렬 생성
# 평가 점수가 없는 항목은 0으로 채워 넣기!
user_item_matrix = user_item_df.pivot_table(index='user_id', columns='cellphone_id', values='rating').fillna(0)
user_item_matrix

cellphone_id,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,10.0,0.0,3.0,0.0,0.0,2.0,3.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,10.0,7.0,0.0,0.0,8.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,7.0,6.0
6,0.0,2.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,0.0,...,8.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,6.0,10.0
8,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,8.0,0.0,6.0,0.0,0.0,9.0
10,0.0,0.0,0.0,9.0,9.0,3.0,7.0,0.0,0.0,0.0,...,9.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,0.0,5.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,18.0,0.0,0.0,0.0,5.0,0.0,8.0
255,0.0,10.0,0.0,0.0,10.0,0.0,10.0,0.0,10.0,10.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
256,0.0,0.0,7.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,7.0,9.0


## 사용자 간 유사도 추출

In [17]:
# 코사인 유사도 계산
user_similarity = cosine_similarity(user_item_matrix)
print('user_similarity matrix 크기 : ', user_similarity.shape)

# 유사도 행렬을 DataFrame으로 변환
user_similarity = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
display(user_similarity)

user_similarity matrix 크기 :  (99, 99)


user_id,0,1,6,8,10,12,16,24,25,26,...,245,246,251,252,253,254,255,256,257,258
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.374194,0.128903,0.026851,0.354701,0.389305,0.077902,0.521537,0.198117,0.326360,...,0.055782,0.090959,0.212096,0.090596,0.223568,0.119819,0.108084,0.058456,0.045584,0.080861
1,0.374194,1.000000,0.538646,0.099237,0.343340,0.224116,0.191944,0.451668,0.394117,0.491044,...,0.222335,0.197222,0.332217,0.206312,0.722997,0.141331,0.199734,0.538317,0.179703,0.153116
6,0.128903,0.538646,1.000000,0.271362,0.254505,0.496402,0.148103,0.107795,0.464910,0.275228,...,0.114980,0.301096,0.135538,0.090059,0.309237,0.205272,0.225630,0.333401,0.079298,0.299196
8,0.026851,0.099237,0.271362,1.000000,0.190702,0.292687,0.211380,0.273634,0.240534,0.486753,...,0.511170,0.394895,0.520372,0.364576,0.140859,0.279952,0.179417,0.371347,0.521912,0.468455
10,0.354701,0.343340,0.254505,0.190702,1.000000,0.382074,0.244531,0.297653,0.248382,0.441223,...,0.344580,0.255408,0.124506,0.307098,0.302199,0.261894,0.419465,0.261893,0.222000,0.331589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,0.119819,0.141331,0.205272,0.279952,0.261894,0.273922,0.280501,0.146491,0.469548,0.175130,...,0.050124,0.136961,0.238034,0.029955,0.179745,1.000000,0.188693,0.181780,0.147554,0.205867
255,0.108084,0.199734,0.225630,0.179417,0.419465,0.201973,0.270281,0.179156,0.240391,0.321749,...,0.227691,0.151480,0.322381,0.381000,0.285774,0.188693,1.000000,0.270418,0.393625,0.096979
256,0.058456,0.538317,0.333401,0.371347,0.261893,0.261708,0.219267,0.316522,0.463169,0.697727,...,0.527466,0.000000,0.667099,0.302220,0.338553,0.181780,0.270418,1.000000,0.370651,0.267870


## 유사도 기반 아이템 추천

In [18]:
def recommend_for_user(user_id, user_similarity, user_item_matrix, top_n=5):
    # 현재 사용자와 다른 사용자 간의 유사도 가져오기
    user_similarities = user_similarity.loc[user_id]

    # 가장 유사한 사용자들 찾기
    similar_users = user_similarities.sort_values(ascending=False)[1:].index

    # 유사한 사용자들이 높게 평가한 항목 추천
    recommended_items = pd.Series(dtype='float64')
    for similar_user in similar_users:
        # 유사한 사용자의 평가 가져오기
        user_ratings = user_item_matrix.loc[similar_user]

        # 현재 사용자가 평가하지 않은 항목만 선택
        user_ratings = user_ratings[user_ratings.index.isin(user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] == 0].index)]

        # 추천 목록에 추가
        recommended_items = recommended_items.add(user_ratings, fill_value=0)

    # 가중치가 높은 순으로 항목 정렬 및 상위 N개 추천
    recommended_items = recommended_items.sort_values(ascending=False)[:top_n]

    return recommended_items.index.tolist()

In [20]:
# 비교의 중심이 되는 베이스 사용자의 index 설정
order_of_user_idx = 8
base_user_idx = user_item_matrix.index.tolist()[order_of_user_idx]

recommendations_for_user = recommend_for_user(base_user_idx, user_similarity, user_item_matrix, top_n=3)
recommendations_for_user

[25, 4, 15]

In [21]:
# 베이스 사용자가 평가한 핸드폰과 해당 점수를 보여주는 코드
base_user_ratings = user_item_df[user_item_df['user_id'] == base_user_idx]

# 베이스 사용자가 평가한 핸드폰의 상세 정보를 가져오기. 원본 핸드폰 데이터와 병합
base_user_rated_phones = pd.merge(base_user_ratings, data_df, left_on='cellphone_id', right_on='cellphone_id', how='left')

# 베이스 사용자가 평가한 핸드폰 및 점수 출력
print(f"베이스 사용자({base_user_idx})가 평가한 핸드폰과 점수:")
display(base_user_rated_phones[['brand', 'model', 'operating system', 'internal memory', 'RAM', 'price', 'rating']])

# 추천 결과로 나온 핸드폰의 스팩 보여주기
# 추천된 핸드폰 ID 리스트를 기반으로 원본 핸드폰 데이터에서 해당 핸드폰의 상세 정보를 가져옴
recommended_phones_info = data_df[data_df['cellphone_id'].isin(recommendations_for_user)]

# 등수 정보를 포함
rankings = [rank + 1 for rank, _ in enumerate(recommendations_for_user)]

# 추천된 핸드폰 ID와 등수를 DataFrame으로 변환
recommended_phones_with_rank = pd.DataFrame({
    'cellphone_id': recommendations_for_user,
    'rank': rankings
})

# 추천된 핸드폰의 상세 정보와 등수 정보를 병합
recommended_phones_info = pd.merge(recommended_phones_info, recommended_phones_with_rank, on='cellphone_id')


# 추천된 핸드폰의 상세 정보 출력
print("\n추천된 핸드폰의 상세 정보:")
display(recommended_phones_info[['cellphone_id', 'brand', 'model', 'operating system', 'internal memory', 'RAM', 'price', 'rank']])


베이스 사용자(25)가 평가한 핸드폰과 점수:


,brand,model,operating system,internal memory,RAM,price,rating
0,Apple,iPhone 13,iOS,128,4,699,9
1,Apple,iPhone XR,iOS,64,3,236,6
2,Samsung,Galaxy Z Fold 3,Android,256,12,1799,2
3,Motorola,Moto G Play (2021),Android,32,3,159,5
4,Samsung,Galaxy S22 Ultra,Android,128,8,840,3
5,Motorola,Moto G Power (2022),Android,64,4,189,9
6,Samsung,Galaxy A32,Android,64,4,199,10
7,Samsung,Galaxy Z Flip 3,Android,128,8,899,1
8,Google,Pixel 6a,Android,128,6,449,9
9,OnePlus,10 Pro,Android,128,8,780,5



추천된 핸드폰의 상세 정보:


,cellphone_id,brand,model,operating system,internal memory,RAM,price,rank
0,4,Apple,iPhone 13 Pro Max,iOS,256,6,1199,2
1,15,Google,Pixel 6,Android,128,8,499,3
2,25,Xiaomi,11T Pro,Android,256,8,435,1


# 아이템 기반 협업 필터링

## 상호 작용 데이터 준비

In [22]:
# 아이템-사용자 평가 행렬 생성
item_user_matrix = user_item_matrix.T

## 아이템 간 유사도 계산

In [23]:
# 코사인 유사도 계산
item_similarity = cosine_similarity(item_user_matrix)

# 유사도 행렬을 DataFrame으로 변환
item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix.index, columns=item_user_matrix.index)

In [24]:
item_similarity_df

cellphone_id,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
cellphone_id,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.273976,0.244768,0.215061,0.175059,0.227267,0.146073,0.231051,0.312913,0.234146,...,0.226938,0.415472,0.207126,0.137457,0.195815,0.116289,0.193472,0.267208,0.059663,0.241137
1,0.273976,1.000000,0.266560,0.197991,0.254580,0.387912,0.134468,0.230430,0.327576,0.389812,...,0.154386,0.131872,0.274997,0.191730,0.150717,0.081416,0.158947,0.159534,0.171223,0.208484
2,0.244768,0.266560,1.000000,0.221172,0.217693,0.263274,0.187792,0.238114,0.232984,0.317820,...,0.099636,0.312116,0.320475,0.305079,0.182127,0.185405,0.170411,0.348632,0.347486,0.208889
3,0.215061,0.197991,0.221172,1.000000,0.318130,0.262854,0.240176,0.171789,0.309219,0.166034,...,0.350881,0.230128,0.221620,0.229334,0.164511,0.205939,0.267679,0.187430,0.232055,0.236037
4,0.175059,0.254580,0.217693,0.318130,1.000000,0.253047,0.291672,0.287327,0.362129,0.253068,...,0.126387,0.256540,0.488881,0.228817,0.364388,0.179135,0.129896,0.209108,0.113544,0.299880
5,0.227267,0.387912,0.263274,0.262854,0.253047,1.000000,0.084891,0.142953,0.327236,0.285440,...,0.101428,0.374987,0.362825,0.364656,0.204691,0.268618,0.272205,0.328921,0.118611,0.347249
6,0.146073,0.134468,0.187792,0.240176,0.291672,0.084891,1.000000,0.117492,0.238082,0.303679,...,0.372618,0.358996,0.362659,0.120058,0.436529,0.171823,0.204458,0.271945,0.199051,0.232839
7,0.231051,0.230430,0.238114,0.171789,0.287327,0.142953,0.117492,1.000000,0.399930,0.295417,...,0.241334,0.309385,0.196169,0.183132,0.148272,0.445269,0.287452,0.328043,0.213677,0.269563
8,0.312913,0.327576,0.232984,0.309219,0.362129,0.327236,0.238082,0.399930,1.000000,0.248853,...,0.200687,0.434765,0.299602,0.215053,0.256624,0.290083,0.331252,0.544958,0.251219,0.377081


## 추천 점수 계산

In [25]:
# 사용자가 평가하지 않은 항목에 대해 추천 점수를 계산하는 함수
def calculate_recommendation_scores(user_id, item_similarity_df, user_item_matrix, topn=5):
    # 사용자가 평가한 항목 확인
    user_ratings = user_item_matrix.loc[user_id]
    rated_items = user_ratings[user_ratings > 0].index

    # 사용자가 평가하지 않은 항목 확인
    unrated_items = user_ratings[user_ratings == 0].index

    # 추천 점수를 저장할 빈 Series 생성
    recommendation_scores = pd.Series(index=unrated_items, dtype=np.float64)

    # 사용자가 평가하지 않은 각 항목에 대해 추천 점수 계산
    for item in unrated_items:
        # 현재 항목과 사용자가 평가한 항목들 간의 유사도
        item_similarities = item_similarity_df.loc[item, rated_items]

        # 유사도와 사용자의 평가 점수의 가중합 계산
        weighted_scores = item_similarities * user_ratings[rated_items]

        # 가중합의 총합을 추천 점수로 사용
        recommendation_scores[item] = weighted_scores.sum() / item_similarities.sum()

    # NaN 값을 제거하고 점수가 높은 순으로 정렬
    recommendation_scores = recommendation_scores.dropna().sort_values(ascending=False)

    return recommendation_scores[:topn]

In [26]:
# 비교의 중심이 되는 베이스 사용자의 index 설정
order_of_user_idx = 8
base_user_idx = user_item_matrix.index.tolist()[order_of_user_idx]

recommendation_scores = calculate_recommendation_scores(base_user_idx, item_similarity_df, user_item_matrix, topn=3)
recommendation_scores

cellphone_id
31    6.497395
0     6.418619
28    6.310959
dtype: float64

In [27]:
# 베이스 사용자의 평가한 핸드폰과 해당 점수를 보여주는 코드
base_user_ratings = user_item_df[user_item_df['user_id'] == base_user_idx]

# 베이스 사용자가 평가한 핸드폰의 상세 정보를 가져오기. 원본 핸드폰 데이터와 병합
base_user_rated_phones = pd.merge(base_user_ratings, data_df, left_on='cellphone_id', right_on='cellphone_id', how='left')

# 베이스 사용자가 평가한 핸드폰 및 점수 출력
print(f"베이스 사용자({base_user_idx})가 평가한 핸드폰과 점수:")
display(base_user_rated_phones[['brand', 'model', 'operating system', 'internal memory', 'RAM', 'price', 'rating']])

# 추천된 핸드폰의 상세 정보를 보여주는 부분에 추천 점수를 추가하기 위한 수정
# 추천 점수를 DataFrame으로 변환하여 상세 정보와 병합
recommendation_scores_df = recommendation_scores.reset_index()
recommendation_scores_df.columns = ['cellphone_id', 'recommendation_score']

# 추천된 핸드폰의 상세 정보와 추천 점수 정보를 병합
recommended_phones_info_with_scores = pd.merge(data_df, recommendation_scores_df, on='cellphone_id')

# 등수 정보를 포함
recommended_phones_info_with_scores['rank'] = recommended_phones_info_with_scores['recommendation_score'].rank(ascending=False)

# 결과 출력
print("\n추천된 핸드폰의 상세 정보와 추천 점수:")
display(recommended_phones_info_with_scores[['cellphone_id', 'brand', 'model', 'operating system', 'internal memory', 'RAM', 'price', 'recommendation_score', 'rank']])

베이스 사용자(25)가 평가한 핸드폰과 점수:


,brand,model,operating system,internal memory,RAM,price,rating
0,Apple,iPhone 13,iOS,128,4,699,9
1,Apple,iPhone XR,iOS,64,3,236,6
2,Samsung,Galaxy Z Fold 3,Android,256,12,1799,2
3,Motorola,Moto G Play (2021),Android,32,3,159,5
4,Samsung,Galaxy S22 Ultra,Android,128,8,840,3
5,Motorola,Moto G Power (2022),Android,64,4,189,9
6,Samsung,Galaxy A32,Android,64,4,199,10
7,Samsung,Galaxy Z Flip 3,Android,128,8,899,1
8,Google,Pixel 6a,Android,128,6,449,9
9,OnePlus,10 Pro,Android,128,8,780,5



추천된 핸드폰의 상세 정보와 추천 점수:


,cellphone_id,brand,model,operating system,internal memory,RAM,price,recommendation_score,rank
0,0,Apple,iPhone SE (2022),iOS,128,4,429,6.418619,2.0
1,28,Sony,Xperia Pro,Android,512,12,1998,6.310959,3.0
2,31,Motorola,Moto G Pure,Android,32,3,129,6.497395,1.0
